In [1]:
import fastf1

fastf1.Cache.enable_cache('cache')

import pandas as pd
import numpy as np
import traceback
from sklearn.linear_model import LinearRegression


In [2]:
def feature_engineering(df):
    
    columns = [
        'Year',
        'Session',
        'Driver',
        'DriverNumber',
        'LapTime',
        'LapNumber',
        'Stint',
        'PitOutTime',
        'PitInTime',
        'Compound',
        'TyreLife',
        'TrackTemp'
    ]
    
    df = df[columns].copy()

    # Correct compound mapping
    df["Compound"] = df["Compound"].map({"HARD": 1, "MEDIUM": 2, "SOFT": 3})

    # Convert laptimes
    df["LapSeconds"] = df["LapTime"].dt.total_seconds()

    # Remove pit laps
    df = df[df["PitInTime"].isna() & df["PitOutTime"].isna()].copy()

    # Per-stint median pace
    df["StintMedian"] = df.groupby(["Driver", "Stint", "Session"])["LapSeconds"].transform("median")

    # Drop slow laps
    df = df[df["LapSeconds"] <= df["StintMedian"] + 1.5]

    df = df.sort_values(["Driver", "Year", "Session", "LapNumber"]).reset_index(drop=True)

    # Remove short stints (<3 laps)
    df["StintLength"] = df.groupby(["Driver", "Session", "Stint"])["LapSeconds"].transform("size")
    df = df[df["StintLength"] >= 3].copy()

    df = df.drop(columns=["PitInTime", "PitOutTime", "LapTime"])

    # ---------- Build stint_df ----------
    stint_rows = []

    for (driver, session, stint), g in df.groupby(["Driver", "Session", "Stint"]):

        X = g["LapNumber"].values.reshape(-1, 1)
        y = g["LapSeconds"].values
        model = LinearRegression().fit(X, y)

        stint_rows.append({
            "Driver": driver,
            "Session": session,
            "Stint": stint,
            "Compound": g["Compound"].iloc[0],
            "TyreLifeStart": g["TyreLife"].min(),
            "TyreLifeEnd": g["TyreLife"].max(),
            "AvgLap": g["LapSeconds"].mean(),
            "DegSlope": model.coef_[0],
            "StintLength": len(g),
            "AvgTrackTemp": g["TrackTemp"].mean(),
            "Year": g["Year"].iloc[0]
        })

    stint_df = pd.DataFrame(stint_rows)

    # ---------- Build driver-level features ----------
    features = []

    for driver, g in stint_df.groupby("Driver"):

        row = {
            "Driver": driver,
            "Year": g["Year"].iloc[0],
            "NumStints": len(g),
            "NumCompoundsUsed": g["Compound"].nunique(),
            "LongestStint": g["StintLength"].max(),
            "AvgPaceSoft": g[g["Compound"] == 3]["AvgLap"].mean(),
            "AvgPaceMedium": g[g["Compound"] == 2]["AvgLap"].mean(),
            "AvgPaceHard": g[g["Compound"] == 1]["AvgLap"].mean(),
            "DegSoft": g[g["Compound"] == 3]["DegSlope"].mean(),
            "DegMed": g[g["Compound"] == 2]["DegSlope"].mean(),
            "AvgTrackTemp": g["AvgTrackTemp"].mean(),
            "MaxTrackTemp": g["AvgTrackTemp"].max(),
            "MinTrackTemp": g["AvgTrackTemp"].min(),
            "DegHard": g[g["Compound"] == 1]["DegSlope"].mean()
        }

        features.append(row)

    return pd.DataFrame(features)



In [3]:
def get_race_list(year):
    schedule = fastf1.get_event_schedule(year, include_testing=False)


    return list(schedule["EventName"])


In [4]:
all_driver_rows = []
years = [2022, 2023, 2024,2025] 

In [ ]:
for year in years:
    races = get_race_list(year)
    for gp in races:
        print(f"\nProcessing {gp} {year}")

        try:
            # Load all 3 practice sessions
            fp1 = fastf1.get_session(year, gp, "FP1")
            fp1.load()
            weather1 = fp1.weather_data[["Time", "TrackTemp"]].copy()
            laps_fp1 = pd.merge_asof(
                fp1.laps.copy().sort_values("LapStartTime"),
                weather1.sort_values("Time"),
                left_on="LapStartTime",
                right_on="Time",
                direction="nearest"
            )
            laps_fp1["Year"] = year
            laps_fp1["Session"] = 1

            try:
                fp2 = fastf1.get_session(year, gp, "FP2")
                fp2.load()
                weather2 = fp2.weather_data[["Time", "TrackTemp"]].copy()
                laps_fp2 = pd.merge_asof(
                    fp2.laps.copy().sort_values("LapStartTime"),
                    weather2.sort_values("Time"),
                    left_on="LapStartTime",
                    right_on="Time",
                    direction="nearest"
                )
                laps_fp2["Year"] = year
                laps_fp2["Session"] = 2
                fp2_available = True
            except:
                print(f"FP2 missing → skipping FP2 for {gp} {year}")
                fp2_available = False
            # Try FP3 — may not exist on sprint weekends
            try:
                fp3 = fastf1.get_session(year, gp, "FP3")
                fp3.load()
                weather3 = fp3.weather_data[["Time", "TrackTemp"]].copy()
                laps_fp3 = pd.merge_asof(
                    fp3.laps.copy().sort_values("LapStartTime"),
                    weather3.sort_values("Time"),
                    left_on="LapStartTime",
                    right_on="Time",
                    direction="nearest"
                )
                laps_fp3["Year"] = year
                laps_fp3["Session"] = 3
                fp3_available = True
            except:
                print(f"FP3 missing → skipping FP3 for {gp} {year}")
                fp3_available = False

            # Combine the 3 sessions into one df
            if fp3_available:
                all_laps = pd.concat([laps_fp1, laps_fp2, laps_fp3], ignore_index=True)
            else:
                all_laps = pd.concat([laps_fp1, laps_fp2], ignore_index=True)

            all_laps["GP"] = gp

            # ★ Feed straight into your FE function
            driver_df = feature_engineering(all_laps)

            # Add GP + year (if not inside your function)
            driver_df["GP"] = gp
            driver_df["Year"] = year

            # Append to master list
            all_driver_rows.append(driver_df)



        except Exception as e:
            print(f"\n--- ERROR in {gp} {year} ---")
            traceback.print_exc()
            continue

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...



Processing Bahrain Grand Prix 2022


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '3', '4', '6', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '47', '55', '63', '77']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 2 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data



Processing Saudi Arabian Grand Prix 2022


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '3', '4', '6', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '47', '55', '63', '77']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Practice 2 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data f

...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '43', '44', '55', '63', '81', '87']
events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 1 [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


FP2 missing → skipping FP2 for Qatar Grand Prix 2025
FP3 missing → skipping FP3 for Qatar Grand Prix 2025

Processing Abu Dhabi Grand Prix 2025


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger      WARNING 	Failed to load timing data!
req            INFO


--- ERROR in Abu Dhabi Grand Prix 2025 ---


Traceback (most recent call last):
  File "C:\Users\thoma\AppData\Local\Temp\ipykernel_13756\987643076.py", line 10, in <module>
    weather1 = fp1.weather_data[["Time", "TrackTemp"]].copy()
               ^^^^^^^^^^^^^^^^
  File "c:\Users\thoma\Digital_Futures\.venv\Lib\site-packages\fastf1\core.py", line 1311, in weather_data
    return self._get_property_warn_not_loaded('_weather_data')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thoma\Digital_Futures\.venv\Lib\site-packages\fastf1\core.py", line 1250, in _get_property_warn_not_loaded
    raise DataNotLoadedError(
fastf1.core.DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`


In [6]:
final_df = pd.concat(all_driver_rows, ignore_index=True)

final_df.sample(20)

,Driver,Year,NumStints,NumCompoundsUsed,LongestStint,AvgPaceSoft,AvgPaceMedium,AvgPaceHard,DegSoft,DegMed,AvgTrackTemp,MaxTrackTemp,MinTrackTemp,DegHard,GP
634,LEC,2023,5,3,10,78.420833,77.609300,68.928208,1.188107,-0.079429,42.672333,48.000000,33.370000,-0.094553,Austrian Grand Prix
464,VER,2022,1,1,6,NaN,90.477000,NaN,NaN,0.072671,34.883333,34.883333,34.883333,NaN,Abu Dhabi Grand Prix
656,NOR,2023,5,3,9,91.601000,91.828833,92.249500,0.523029,-0.100750,40.093333,41.600000,37.566667,-0.121475,British Grand Prix
1603,GAS,2025,9,2,11,80.156593,77.730773,NaN,-1.200375,-1.134604,42.137744,45.720000,36.936364,NaN,Canadian Grand Prix
1517,ANT,2025,2,1,13,NaN,90.999356,NaN,NaN,-0.154567,41.015385,44.230769,37.800000,NaN,Miami Grand Prix
50,OCO,2022,6,3,4,82.974056,83.299750,83.999083,-0.070817,-0.428051,33.452778,38.150000,28.275000,-0.392075,Australian Grand Prix
352,GAS,2022,2,2,3,92.992000,96.566333,NaN,0.051000,0.798000,29.850000,30.066667,29.633333,NaN,Japanese Grand Prix
1536,ALO,2025,4,2,5,80.507311,NaN,76.666000,0.064361,NaN,39.605000,41.933333,36.333333,-0.136816,Emilia Romagna Grand Prix
1226,LEC,2024,2,1,11,NaN,85.893485,NaN,NaN,0.221509,48.945455,51.800000,46.090909,NaN,Italian Grand Prix
490,DEV,2023,5,3,7,93.891179,93.810200,92.289667,-0.059452,-0.447500,35.562381,44.000000,30.828571,-0.377500,Saudi Arabian Grand Prix


In [7]:
final_df.shape

(1898, 15)

In [14]:
feature_driver_counts = (
    final_df.groupby(["Year", "GP"])["Driver"]
    .nunique()
    .reset_index(name="DriverCount")
    .sort_values(["Year", "GP"])
)

feature_driver_counts[feature_driver_counts["DriverCount"] != 20]


,Year,GP,DriverCount
0,2022,Abu Dhabi Grand Prix,28
5,2022,Belgian Grand Prix,21
10,2022,French Grand Prix,22
11,2022,Hungarian Grand Prix,21
12,2022,Italian Grand Prix,22
14,2022,Mexico City Grand Prix,25
19,2022,Spanish Grand Prix,23
20,2022,São Paulo Grand Prix,21
21,2022,United States Grand Prix,24
22,2023,Abu Dhabi Grand Prix,30


In [8]:
final_df.to_csv("f1_practice_features.csv", index=False)
